In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from mll_calc.mll_pred import format_XY, mll_testset

## VAN-2|EF05|WZR0058|E58-88 Simulation Parameters:
* PWR: Vandellos-2_PWR, so origen equivalent is W(17x17)
* Enrichment: 4.5
* Burnup: 43520.0 MW*d/tHM
* Cooling time: 2000 - 4000 days
* U weight: 72.45 MT (Did 1 MTU in sim)
* Thermal power: 2940 MW for cycles 7-10, and 3070 MW for cycle 11
* Avg power for ORIGEN-ARP is 40.58 & 42.37 MW/MTHM-- did avg of 41 for sim

In [2]:
train_db = '~/sims_n_results/sfcompo_compare_aug2020/van2ohi2_not-scaled_15nuc_5.pkl'
test_db = '~/sfcompo/format_clean/sfcompo_formatted.pkl'
XY = format_XY(train_db)
XY = XY[0:6]
test = pd.read_pickle(test_db)

In [3]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
unc = 0.05
lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
nonlbls = ['AvgPowerDensity', 'ModDensity', 'UiWeight']

### Test sample from SFCOMPO that was simulated in ORIGEN

#### All values are mg/gUi

In [4]:
test = test.loc[test.index == 'VAN-2|EF05|WZR0058|E58-88']
test

,Burnup,CoolingTime,Enrichment,OrigenReactor,ReactorType,ba136,ba138,cs133,cs134,cs135,cs137,eu153,eu154,pu239,pu240,pu241,pu242,sm149,sm150,sm152
VAN-2|EF05|WZR0058|E58-88,43520.0,0.0,4.5,Vandellos-2_PWR,pwr,0.0,0.0,1.4,0.0184,0.784,1.43,0.133,0.0166,6.14,2.4,1.13,0.556,0.00339,0.331,0.125


### ORIGEN Sims

#### All values are g --> converting to mg/gUi
Dividing all values by 1000. 


[x (g) / 1e6 (gUi)] * [1000 (mg) / 1 (g)] = x / 1000

In [5]:
isos = XY.columns[~XY.columns.isin(lbls+nonlbls)].tolist()
print(isos)

['ba136', 'ba138', 'cs133', 'cs134', 'cs135', 'cs137', 'eu153', 'eu154', 'pu239', 'pu240', 'pu241', 'pu242', 'sm149', 'sm150', 'sm152']


In [6]:
XY[isos] = XY[isos].div(1000, axis=0)

In [7]:
XY

,AvgPowerDensity,Burnup,CoolingTime,Enrichment,ModDensity,OrigenReactor,ReactorType,UiWeight,ba136,ba138,cs133,cs134,cs135,cs137,eu153,eu154,pu239,pu240,pu241,pu242,sm149,sm150,sm152
0,41,43520.0,0.0,4.5,0.71,w17x17,pwr,1,0.02548,1.742,1.468,0.185100,0.5289,1.622,0.1561,0.03729,5.850,1.680,1.962,0.7503,0.003038,0.4006,0.1251
1,41,43520.0,2000.0,4.5,0.71,w17x17,pwr,1,0.02621,1.742,1.480,0.029460,0.5296,1.430,0.1572,0.02399,5.933,1.691,1.504,0.7503,0.004398,0.4006,0.1251
2,41,43520.0,2500.0,4.5,0.71,w17x17,pwr,1,0.02621,1.742,1.480,0.018610,0.5296,1.385,0.1572,0.02148,5.932,1.694,1.408,0.7503,0.004398,0.4006,0.1251
3,41,43520.0,3000.0,4.5,0.71,w17x17,pwr,1,0.02621,1.742,1.480,0.011750,0.5296,1.342,0.1572,0.01924,5.932,1.696,1.317,0.7503,0.004398,0.4006,0.1251
4,41,43520.0,3500.0,4.5,0.71,w17x17,pwr,1,0.02621,1.742,1.480,0.007424,0.5296,1.301,0.1572,0.01723,5.932,1.698,1.233,0.7503,0.004398,0.4006,0.1251
5,41,43520.0,4000.0,4.5,0.71,w17x17,pwr,1,0.02621,1.742,1.480,0.004689,0.5296,1.260,0.1572,0.01543,5.932,1.700,1.153,0.7503,0.004398,0.4006,0.1251


### First, try to find best prediction via MLL

In [8]:
train = XY.drop(columns=nonlbls)

In [9]:
pred_df = mll_testset(train, test, True, unc, lbls)

In [10]:
pred_df

,sim_idx,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,pred_idx,pred_ReactorType,pred_CoolingTime,pred_Enrichment,pred_Burnup,pred_OrigenReactor,MaxLogLL,MaxLLUnc,2ndMaxLogLL,2ndMaxLLUnc,CDF_LogLL_0.9998,CDF_LLUnc_0.9998,CDF_LogLL_0.9988,CDF_LLUnc_0.9988,CDF_LogLL_0.95,CDF_LLUnc_0.95,CDF_LogLL_0.9,CDF_LLUnc_0.9,CDF_LogLL_0.5,CDF_LLUnc_0.5,CDF_LogLL_0.1,CDF_LLUnc_0.1,CDF_LogLL_0.01,CDF_LLUnc_0.01
0,VAN-2|EF05|WZR0058|E58-88,pwr,0.0,4.5,43520.0,Vandellos-2_PWR,2,pwr,2500.0,4.5,43520.0,w17x17,-92.121344,26.037634,-134.383513,28.18844,-92.163606,238.000899,-92.374917,237.224675,-102.686886,199.344929,-113.252429,160.533714,-239.533848,33.785269,-1149.26119,27.113037,-1722.193448,26.145174


### Second, look at nuclide concentration diffs

In [11]:
def logpdf_calc(sim_row, test_sample, unc):
    y_sim = sim_row[test_sample>0].values.tolist()
    std = sim_row.multiply(unc)[test_sample>0].values.tolist()
    y_mes = test_sample[test_sample>0].values.tolist()
    logpdf_list = stats.norm.logpdf(y_sim, loc=y_mes, scale=std)
    return logpdf_list

In [12]:
test_sample = test.drop(columns=lbls).squeeze()
test_answer = test[lbls].squeeze()
sim_row = train.loc[2].drop(lbls).squeeze()
logpdf = logpdf_calc(sim_row, test_sample, unc)
logpdf = pd.Series(logpdf, index=sim_row[test_sample>0].index, name='LogPDF')

In [13]:
df = pd.DataFrame([test_sample, sim_row])
df.loc['Abs Diff'] = np.abs(df.loc[2] - df.loc['VAN-2|EF05|WZR0058|E58-88'])
df.loc['% Diff'] = df.loc['Abs Diff'] * 100 / df.loc['VAN-2|EF05|WZR0058|E58-88']

In [14]:
df.loc['LogPDF'] = logpdf

In [15]:
df

,ba136,ba138,cs133,cs134,cs135,cs137,eu153,eu154,pu239,pu240,pu241,pu242,sm149,sm150,sm152
VAN-2|EF05|WZR0058|E58-88,0.00000,0.000,1.400000,0.018400,0.784000,1.430000,0.133000,0.016600,6.140000,2.400000,1.130000,0.556000,0.003390,0.33100,0.125000
2,0.02621,1.742,1.480000,0.018610,0.529600,1.385000,0.157200,0.021480,5.932000,1.694000,1.408000,0.750300,0.004398,0.40060,0.125100
Abs Diff,0.02621,1.742,0.080000,0.000210,0.254400,0.045000,0.024200,0.004880,0.208000,0.706000,0.278000,0.194300,0.001008,0.06960,0.000100
% Diff,inf,inf,5.714286,1.141304,32.448980,3.146853,18.195489,29.397590,3.387622,29.416667,24.601770,34.946043,29.734513,21.02719,0.080000
LogPDF,NaN,NaN,1.100384,6.035383,-43.437207,1.539961,-0.812722,-4.405458,0.050535,-33.188921,-6.062139,-11.048300,-3.002679,-3.04549,4.155308
